In [1]:
%config IPCompleter.greedy=True
import numpy as np
import cv2
import random
import operator
import math

In [2]:
class Card:
    def __init__(self):
        
        self.contour = []    #card contour
        self.width = 0       #contour max width?
        self.height = 0      #contour max height?
        self.center = []
        self.coords = []     #the rects 4 corners
        self.warped = []     #warped img of the card
        
        self.num_img = []
        self.suit_img = []
        
        self.num = ""
        self.suit = "Unknown"
        self.suit_id = None # QUICKFIND - could produce a fail using a cam? maybe change to 0
        self.num_id = None
        
        self.valid = True
        
    def draw_warped(self):
        cv2.imshow(self.num+self.suit+str(self.height)+str(self.width), self.warped)
        
    def draw_card_outlines(self, img):
        cv2.polylines(img, np.array([self.coords]), True, (0,255,255), 2)
        
    def draw_suit(self):
        cv2.imshow(str(np.sum(self.suit_img)), self.suit_img)
        
    def draw_num(self):
        cv2.imshow(str(np.sum(self.num_img)), self.num_img)
        
    def draw_text(self, img):
        
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        TEXT = str(self.num) + " " + self.suit
        FONT_SIZE = 0.8
        FONT_COLOR = (0, 255, 0)
        THICKNESS = 1
        
        ######
        
        # get boundary of this text
        textsize = cv2.getTextSize(TEXT, FONT, FONT_SIZE, THICKNESS)[0]

        # get coords based on boundary
        text_x = int(self.center[0] - (textsize[0]/2))
        text_y = int(self.center[1] + (textsize[1]/2))
        
        text_y = text_y - int(self.height/2) - 25
        
        
        # add text centered on image
        cv2.putText(img, TEXT, (text_x, text_y), FONT, FONT_SIZE, FONT_COLOR, thickness=THICKNESS)
    
    def set_center(self):
        
        # adds x's together & y's together and takes the avg
        pts_sum = np.sum(self.coords, axis=0)/len(self.coords)
        
        #convert to int as we cant land between 2 pixels
        x = int(pts_sum[0])
        y = int(pts_sum[1])
        
        self.center = [x, y]
        
        
    def get_warp(self, img):
    
        def warpi(rect):

            rect = np.array(rect, dtype = "float32").reshape((4,2))

            maxWidth = 200
            maxHeight = 300

            dst = np.array([
                [0, 0],
                [maxWidth - 1, 0],
                [maxWidth - 1, maxHeight - 1],
                [0, maxHeight - 1]], dtype = "float32")

            # compute the perspective transform matrix and then apply it
            M = cv2.getPerspectiveTransform(rect, dst)
            warped = cv2.warpPerspective(img, M, (maxWidth, maxHeight))

            return warped


        def compare_warps(warp1, warp2):

            HEIGHT = 50
            WIDTH = 50

            sum1 = np.sum(warp1[0:HEIGHT, 0:WIDTH])
            sum2 = np.sum(warp2[0:HEIGHT, 0:WIDTH])

            sum_lowest = min(sum1,sum2)

            if(sum_lowest == sum1): return warp1
            else: return warp2

            
        cent = self.center

        order_rect = lambda p: math.atan2(p[1]-cent[1],p[0]-cent[0])

        self.coords = sorted(self.coords, key=order_rect)

        rect1 = self.coords # first rect from coords
        rect2 = self.coords[1:] + [self.coords[0]] #second rect is just rect1 but starting from a neigbour point

        self.warped = compare_warps(warpi(rect1), warpi(rect2))

        
    def validate_card(self):
        
        # FOUND BY SLAVE LABOR
        CORNER_WIDTH = 48
        CORNER_HEIGHT = 140
        
        # Taken from comparison imgs
        NUM_WIDTH = 70
        NUM_HEIGHT = 125
        SUIT_WIDTH = 70
        SUIT_HEIGHT = 100
        
        # I found this... by trial and error
        ICON_THRESHHOLD = 180 ## QUICKFIND

        #####
        
        # 1.Cut corner out 2. Make it bigger 3. Make it grayscale
        corner = self.warped[0:CORNER_HEIGHT, 0:CORNER_WIDTH]
        corner_big = cv2.resize(corner, (0,0), fx=3, fy=3)
        corner_big_gray = cv2.cvtColor(corner_big, cv2.COLOR_BGR2GRAY)
        

        #Make it black & white and more crisp
        _, thresh = cv2.threshold(corner_big_gray, ICON_THRESHHOLD, 255, cv2.THRESH_BINARY_INV)
        
        
        #divide img into number & suit
        number = thresh[0:255, 0:144]
        suit = thresh[256:420, 0:144]

        
        def clean_icon(img, new_width, new_height):
    
            #find contours
            contours, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #get the biggest contour by looking at their areas
            contours = sorted(contours, key=cv2.contourArea,reverse=True)

            
            ###################################################################
            ### Check if card is valid by checking if it has a contour icon ###
            ###################################################################
            
            if(len(contours) == 0):
                self.valid = False
                print("self.valid set to false")
                cv2.imshow("test", img)
                return
            
            ###################################################################
            
            #get surrounding rect of contour
            x, y, w, h = cv2.boundingRect(contours[0])

            #extract new tight img of icon 
            rect = img[y:y+h, x:x+w]
            
            #return resized icon
            return cv2.resize(rect, (new_width, new_height), 0, 0)
        
        
        self.num_img = clean_icon(number, NUM_WIDTH, NUM_HEIGHT)
        self.suit_img = clean_icon(suit, SUIT_WIDTH, SUIT_HEIGHT)
    
    def find_suit(self, suit_book):
        
        best_suited_suit = []
        
        for tpl in suit_book:
            
            name = tpl[0]
            diff = cv2.absdiff(self.suit_img, tpl[1])
            diff_sum = np.sum(diff)
            
            best_suited_suit.append( (name,diff_sum) )
            
        best_suited_suit.sort(key = operator.itemgetter(1))
        
        self.suit = best_suited_suit[0][0]
        
        self.suit_id = list(map(lambda x: x[0], suit_book)).index(self.suit) + 1
        
        return best_suited_suit
    
    def find_num(self, num_book):
        
        best_suited_num = []
        
        for tpl in num_book:
            
            name = tpl[0]
            diff = cv2.absdiff(self.num_img, tpl[1])
            diff_sum = np.sum(diff)
            
            best_suited_num.append( (name,diff_sum) )
            
        best_suited_num.sort(key = operator.itemgetter(1))
        
        self.num = best_suited_num[0][0]
        
        self.num_id = list(map(lambda x: x[0], num_book)).index(self.num) + 1
        
        return best_suited_num

In [3]:
def load_icons():
    ###
    suit_names = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    #suit_imgs = {el:cv2.imread("./cards/"+el+".jpg", cv2.IMREAD_GRAYSCALE) for el in suit_names}
    suit_imgs = []
    ###
    num_names = ['Ace','Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack','Queen','King']
    num_imgs = []

    for name in num_names:
        tmp_img = cv2.imread("./new_cards/"+name+".jpg", cv2.IMREAD_GRAYSCALE)
        tmp_tuple = (name, tmp_img)
        num_imgs.append(tmp_tuple)
    
    for name in suit_names:
        tmp_img = cv2.imread("./new_cards/"+name+".jpg", cv2.IMREAD_GRAYSCALE)
        tmp_tuple = (name, tmp_img)
        suit_imgs.append(tmp_tuple)

    return suit_imgs, num_imgs


def preprocess_frame(frame):
    
    BKG_THRESH = 160
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    
    _, thresh = cv2.threshold(blur,BKG_THRESH,255,cv2.THRESH_BINARY)

    return thresh

def find_cards(og_frame, pp_frame):
    
    #List holding valid cards
    found_cards = []
    
    #Find all contours from given frame
    contours, _ = cv2.findContours(pp_frame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #Go through each contour as c
    for c in contours:
        
        #Calculate stuffs
        size = cv2.contourArea(c)
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.01*peri, True)
    
        #print(size)
    
        #Check if valid card (has 4 corners and is realistic size)
        if(len(approx) == 4 and size>4000 and size<100000):
            
            card = Card()
        
            card.contour = c
            
            card.coords = list(approx.reshape(4,2))
            
            _,_, card.width, card.height = cv2.boundingRect(c)
            
            card.set_center()
            
            card.get_warp(og_frame) # extracts a warped image of the card 

            card.validate_card() # inserts the warp into card and generates image of num and suit and check if valid
        
            if card.valid: found_cards.append(card)
        
    return found_cards

def draw_predicted_hand(img, text):

    FONT = cv2.FONT_HERSHEY_SIMPLEX
    TEXT = text
    FONT_SIZE = 1
    FONT_COLOR = (0, 255, 255)
    THICKNESS = 1

    ######

    # get boundary of this text
    text_w, text_h = cv2.getTextSize(TEXT, FONT, FONT_SIZE, THICKNESS)[0]
    
    img_h, img_w, _ = img.shape
    
    text_x = int(img_w/2 - text_w/2)
    text_y = int(img_h - text_h)

    cv2.putText(img, TEXT, (text_x, text_y), FONT, FONT_SIZE, FONT_COLOR, thickness=THICKNESS)


In [4]:

#######################################
### returns list of all cards found ###
#######################################


def process_frame(frame): # returns new frame & list of all cards found
    
    prepross_frame = preprocess_frame(frame) # processes frame to make it easier to find cards

    found_cards = find_cards(frame, prepross_frame) # returns a list of cards from the processed frame

    suit_book, num_book = load_icons()
    
    
    for card in found_cards:
        

        card.draw_card_outlines(frame) # draws card on img

        card.find_suit(suit_book) # returns list of tuples with (suitname, %chance) in order of best match and sets best match in Card

        card.find_num(num_book) # --------

        card.draw_text(frame) # draw text over card
        
        
    if len(found_cards) >= 1:
        nums = [card.num_id for card in found_cards[:5]]
        suits = [card.suit_id for card in found_cards[:5]]
        arr = nums + suits
        
        print(arr)
        
        
    draw_predicted_hand(frame, "Nothing")
    
    
    cv2.imshow("prepross_frame", prepross_frame)
    cv2.imshow('frame', frame)
    
    return found_cards    

In [ ]:
def save_img_icon(img_path, save_path, suit=True):
    
    img = cv2.imread(img_path)
    
    prepross_frame = preprocess_frame(img) # processes frame to make it easier to find cards
    
    found_cards = find_cards(img, prepross_frame) # returns a list of found cards
    
    if(len(found_cards) == 0): return
    
    card = found_cards[0] # take first card (there should only be one)
    
    if suit: cv2.imwrite(save_path, card.suit_img);
    else: cv2.imwrite(save_path, card.num_img);

In [ ]:
def use_img(img):
    img = cv2.imread(img)

    cards = process_frame(img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def use_cam(num=0):
    cap = cv2.VideoCapture(num)

    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'): break
        ret, frame = cap.read()
        cards = process_frame(frame)

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(0)

In [ ]:
use_img("img6.jpg")
#use_cam(1)
#save_img_icon("./pre/clubs.jpg", "./delete/this2.jpg", suit=False)

self.valid set to false
13
8
[13, 8, 1, 1]
